In [ ]:
setTimeout(() => console.log("1秒経過しました"), 3000)
console.log("settimeout 実行")

In [ ]:
import * as fs from "fs"
fs.readdir("foo", (err, files) => {
    console.log("fs.readdir() 実行結果")
    console.log(`err: ${err}`)
    console.log(`files: ${files}`)
})

In [ ]:
function parseJSONSync(json){
    try{
        return JSON.parse(json)
    }
    catch(err){
        console.log("例外発生：", err)
    }
}

In [ ]:
parseJSONSync('{"message": "Hello", "to": "World"}')

In [ ]:
parseJSONSync('不正')

In [ ]:
function parseJSONSync2(json, callback){
    try{
        setTimeout(() => {
            callback(JSON.parse(json))
        }, 2000)
    }
    catch(err){
        console.log("例外発生：", err)
        callback({})
    }
}

In [ ]:
parseJSONSync2('{"message": "Hello", "to": "World"}', result => console.log('parse結果:', result))

In [ ]:
parseJSONSync2('不正', result => console.log('不正:', result))

In [2]:
function parseJSONSync3(json, callback){
    setTimeout(() => {
        try{
            callback(JSON.parse(json))
        }
        catch(err){
            console.log("例外発生：", err)
            callback(err)
        }
    }, 2000)
}

In [ ]:
parseJSONSync3('{"message": "Hello", "to": "World"}', result => console.log('parse結果:', result))

In [ ]:
parseJSONSync3('不正', result => console.log('不正:', result))

In [ ]:
const cache = {}
function parseJSONAsyncWithCache(json, callback){
    const cached = cache[json]
    if(cached){
        callback(cached.err, cached.result)
        return
    }
    parseJSONSync3(json, (err, result) => {
        cache[json] = {err, result}
        callback(err, result)
    })
}

In [ ]:
parseJSONAsyncWithCache('{"message": "Hello", "to": "World"}', result => {
    console.log('1回目parse結果:', result)
    parseJSONAsyncWithCache('{"message": "Hello", "to": "World"}', result => {
        console.log('2回目parse結果:', result)
    })
    console.log("2回目完了")
})
console.log("1回目完了")

In [3]:
const cache2 = {}
function parseJSONAsyncWithCache2(json, callback){
    const cached = cache2[json]
    if(cached){
        setTimeout(() => {
            callback(cached.err, cached.result)
        }, 0); 
        return
    }
    parseJSONSync3(json, (err, result) => {
        cache2[json] = {err, result}
        callback(err, result)
    })
}

In [4]:
parseJSONAsyncWithCache2('{"message": "Hello", "to": "World"}', result => {
    console.log('1回目parse結果:', result)
    parseJSONAsyncWithCache2('{"message": "Hello", "to": "World"}', result => {
        console.log('2回目parse結果:', result)
    })
    console.log("2回目完了")
})
console.log("1回目完了")

1回目完了
1回目parse結果: { message: 'Hello', to: 'World' }
2回目完了
2回目parse結果: { message: 'Hello', to: 'World' }


In [7]:
function parseJSONAsync4(json){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            try{
                resolve(JSON.parse(json))
            }
            catch(err){
                console.log("例外発生：", err)
                reject(err)
            }
        }, 2000)
    })
}

In [8]:
const toBeFullfilled = parseJSONAsync4('{"message": "Hello", "to": "World"}')
const toBeRejected = parseJSONAsync4('不正')
console.log("* Promise生成直後 ****************************")
console.log(toBeFullfilled)
console.log(toBeRejected)

setTimeout(() => {
    console.log("* 2秒後 ****************************")
    console.log(toBeFullfilled)
    console.log(toBeRejected)
}, 2000)

* Promise生成直後 ****************************
Promise { <pending> }
Promise { <pending> }
Timeout {
  _idleTimeout: 2000,
  _idlePrev: [TimersList],
  _idleNext: [Timeout],
  _idleStart: 8904941,
  _onTimeout: [Function (anonymous)],
  _timerArgs: undefined,
  _repeat: null,
  _destroyed: false,
  [Symbol(refed)]: true,
  [Symbol(kHasPrimitive)]: false,
  [Symbol(asyncId)]: 155,
  [Symbol(triggerId)]: 0
}
例外発生： SyntaxError: Unexpected token 不 in JSON at position 0
    at JSON.parse (<anonymous>)
    at Timeout._onTimeout (evalmachine.<anonymous>:7:30)
    at listOnTimeout (node:internal/timers:557:17)
    at processTimers (node:internal/timers:500:7)


UnhandledPromiseRejection: SyntaxError: Unexpected token 不 in JSON at position 0
    at JSON.parse (<anonymous>)
    at Timeout._onTimeout (evalmachine.<anonymous>:7:30)
    at listOnTimeout (node:internal/timers:557:17)
    at processTimers (node:internal/timers:500:7)


* 2秒後 ****************************
Promise { { message: 'Hello', to: 'World' } }
Promise {
  <rejected> SyntaxError: Unexpected token 不 in JSON at position 0
      at JSON.parse (<anonymous>)
      at Timeout._onTimeout (evalmachine.<anonymous>:7:30)
      at listOnTimeout (node:internal/timers:557:17)
      at processTimers (node:internal/timers:500:7)
}


In [19]:
const stringPromise = Promise.resolve("hello world")
const stringPromise2 = new Promise<string>(resolve => resolve("hello world"))

In [21]:
const numberPromise = stringPromise2.then(str => str.length)
numberPromise

Promise { <pending> }


In [22]:
numberPromise

Promise { 11 }


In [59]:
function asyncFunc1(){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            try{
                console.log("inner fulfilled1")
                resolve("fulfill: asyncFunc1")
            }
            catch(err){
                reject(err)
            }
        }, 2000)
    })
}

function asyncFunc2(){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            try{
                throw new Error("エラー")
                console.log("inner fulfilled2")
                resolve("fulfill: asyncFunc2")
            }
            catch(err){
                reject(err)
            }
        }, 2000)
    })
}

function asyncFunc3(){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            try{
                console.log("inner fulfilled3")
                resolve("fulfill: asyncFunc3")
            }
            catch(err){
                reject(err)
            }
        }, 2000)
    })
}

In [60]:
const ret = asyncFunc1()
    .then(ret => console.log("call", ret))
    .then(asyncFunc2)
    .then(ret => console.log("call", ret))
    .then(asyncFunc3)
    .then(ret => console.log("call", ret))
    .catch(err => console.log(err))
    .finally(() => console.log("close"))

inner fulfilled1
call fulfill: asyncFunc1
Error: エラー
    at Timeout._onTimeout (evalmachine.<anonymous>:21:23)
    at listOnTimeout (node:internal/timers:557:17)
    at processTimers (node:internal/timers:500:7)
close


In [52]:
ret

Promise { undefined }


In [70]:
const somePromise = asyncFunc2()
somePromise.then(
    value => {
        console.log("fulfilled: ", value)
    },
    err => {
        console.log("rejected: ", err)
    }
)
.catch(err => {
    console.log("catch: ", err)
})

Promise { <pending> }
rejected:  Error: エラー
    at Timeout._onTimeout (evalmachine.<anonymous>:21:23)
    at listOnTimeout (node:internal/timers:557:17)
    at processTimers (node:internal/timers:500:7)


In [71]:
somePromise

Promise {
  <rejected> Error: エラー
      at Timeout._onTimeout (evalmachine.<anonymous>:21:23)
      at listOnTimeout (node:internal/timers:557:17)
      at processTimers (node:internal/timers:500:7)
}
